# Session 3 – UK Respiratory Deaths

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/ShamsaraE/time-series-medicine-biology-2026/blob/main/notebooks/03_uk_respiratory_deaths_Deterministic_Stochastic_model.ipynb
)

## Deterministic Seasonality → Stochastic Dependence 


You will learn to:
1. Visualize seasonality and short-term dependence
2. Remove deterministic seasonality (harmonic regression, STL)
3. Diagnose remaining autocorrelation (ACF/PACF, Ljung–Box)





## 0. Imports

We load tools for:
- regression and decomposition
- ACF/PACF diagnostics


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox



## 0.1 Big Picture (Concept Map)

Raw time series typically contains:

$
\text{Data} =
\underbrace{\text{deterministic structure}}_{\text{seasonality/trend}}
+
\underbrace{\text{stochastic structure}}_{\text{autocorrelation}}
+
\underbrace{\text{noise}}_{\text{unpredictable}}
$

**Workflow**
1) plot and inspect  
2) quantify dependence (lag scatter, ACF/PACF)  
3) remove deterministic seasonality  
4) check if residuals look like white noise  
5) if not, fit stochastic model (SARIMA)


## 1. Load Data


We analyze monthly respiratory deaths in the UK (1974–1979) for:
- male (`mdeaths`)
- female (`fdeaths`)



In [ ]:
fdeaths_url = 'https://vincentarelbundock.github.io/Rdatasets/csv/datasets/fdeaths.csv'
mdeaths_url = 'https://vincentarelbundock.github.io/Rdatasets/csv/datasets/mdeaths.csv'

female = pd.read_csv(fdeaths_url, index_col=0)['value']
male = pd.read_csv(mdeaths_url, index_col=0)['value']

dates = pd.date_range(start='1974-01-01', periods=len(male), freq='M')
male.index = dates
female.index = dates

male.head()


### Interpretation
If `male.head()` shows dates like `1974-01-31`, `1974-02-28`, ... the time index is correct.


## 2. Plot the Series

### Why?
Visualization answers the first questions:
- Is there seasonality?
- Is there a trend?
- Is variance constant?


In [ ]:
plt.figure(figsize=(10,4))
plt.plot(male, marker='o', label='Male')
plt.plot(female, marker='o', markerfacecolor='none', label='Female')
plt.title("UK Respiratory Deaths (Male & Female)")
plt.xlabel("Month")
plt.ylabel("Deaths")
plt.legend()
plt.tight_layout()
plt.show()


### Interpretation
Observations in this dataset:

- Clear yearly oscillation → **strong annual seasonality**
- Winter peaks are higher → consistent with respiratory disease patterns
- Male series has higher baseline than female


## 3. Lag-1 Scatter Plot (Visual Autocorrelation)


A lag-1 scatter plots $(X_{t-1}, X_t)$.
If points align along a diagonal → strong lag-1 autocorrelation.

This is a geometric view of $\rho(1)$.


In [ ]:
def lag_scatter(series: pd.Series, title: str):
    x_tm1 = series.values[:-1]
    x_t = series.values[1:]
    plt.figure(figsize=(5,5))
    plt.scatter(x_tm1, x_t)
    plt.xlabel(r"$X_{t-1}$")
    plt.ylabel(r"$X_t$")
    plt.title(title)
    plt.tight_layout()
    plt.show()

lag_scatter(male, "Lag-1 Scatter (Male)")
lag_scatter(female, "Lag-1 Scatter (Female)")


### Interpretation
- A strong diagonal cloud means: high deaths one month tend to be followed by high deaths next month.
- However, this dependence can be partly **seasonal** (winter months follow winter months).

So we next remove deterministic seasonality and re-check dependence.


## 4. Remove Deterministic Seasonality: Harmonic Regression

We model a smooth annual cycle using sine/cosine terms:

\begin{align}
X_t &= \beta_0 + \beta_1 \sin(2\pi t/12) + \beta_2 \cos(2\pi t/12) + \varepsilon_t
\end{align}

### Why?
To remove deterministic seasonal structure so that the residuals represent
the **stochastic component** (plus noise).


In [ ]:
def harmonic_regression(series: pd.Series, period: int = 12):
    t = np.arange(len(series))
    sin_term = np.sin(2 * np.pi * t / period)
    cos_term = np.cos(2 * np.pi * t / period)
    X = np.column_stack([sin_term, cos_term])
    X = sm.add_constant(X)
    model = sm.OLS(series.values, X).fit()
    fitted = model.predict(X)
    resid = series.values - fitted
    return model, fitted, resid

male_harm_model, male_harm_fit, male_harm_resid = harmonic_regression(male, period=12)
female_harm_model, female_harm_fit, female_harm_resid = harmonic_regression(female, period=12)

male_harm_model.summary()


### Interpretation of the Regression Summary

Key lines:

- **R-squared = 0.792** → about **79%** of the variance is explained by a simple annual sinusoid (seasonality dominates).
- **Prob(F-statistic) = 2.93e-24** → overwhelmingly strong evidence that the seasonal terms matter (reject “no seasonality”).
- **Durbin–Watson = 1.253** → residuals still show **positive autocorrelation** (DW < 2).

Conclusion:
Seasonality is captured well, but residuals are not independent → we must analyze stochastic dependence next.



### Seasonal Amplitude

The sine/cosine coefficients can be converted into an amplitude:

\begin{align}
A &= \sqrt{\beta_1^2 + \beta_2^2}
\end{align}

Interpretation:
$A$ is the approximate size of the seasonal swing around the mean level.


In [ ]:
b0, b1, b2 = male_harm_model.params
A = np.sqrt(b1**2 + b2**2)
A


### Interpretation
$A$ is the estimated magnitude of the annual seasonal swing.
For example, if $A\approx 540$, deaths typically fluctuate by roughly ±540 around the mean due to seasonality.


## 4.1 Plot: Original vs Harmonic Seasonal Fit


To visually confirm that the sinusoid tracks the annual peaks/troughs.


In [ ]:
plt.figure(figsize=(10,4))
plt.plot(male.index, male.values, label="Male (original)")
plt.plot(male.index, male_harm_fit, label="Male (harmonic fit)")
plt.title("Harmonic Regression Seasonal Fit (Male)")
plt.legend()
plt.tight_layout()
plt.show()


## Alternatives for Deterministic Seasonality

In the harmonic regression example, we used **one sine/cosine pair**:

$
X_t = \beta_0 + \beta_1 \sin\left(\frac{2\pi t}{s}\right) + \beta_2 \cos\left(\frac{2\pi t}{s}\right) + \varepsilon_t
$

This assumes:

- A single smooth sinusoidal seasonal pattern  
- Fixed amplitude  
- Fixed shape across years  

However, real seasonal patterns are often more complex.

---

## 1 Multiple Fourier Terms (More Flexible Seasonality)

Instead of one sine/cosine pair, we can include multiple harmonics:

$
X_t =
\sum_{k=1}^{K}
\left[
a_k \sin\left(\frac{2\pi k t}{s}\right)
+
b_k \cos\left(\frac{2\pi k t}{s}\right)
\right]
+ \varepsilon_t
$

where:

- \( s \) = seasonal period (12 for monthly data)
- \( K \) = number of harmonics
- Higher \( K \) → more flexible seasonal shape

---

###  Interpretation

- \( K = 1 \) → smooth sinusoid (basic annual wave)
- \( K = 2 \) → allows asymmetric peaks/troughs
- \( K = 3 \) or more → captures sharper or irregular seasonal shapes

This is still a **deterministic seasonal model**, but more flexible than a single harmonic.

---

## Implementation Using `statsmodels`

```python
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
import statsmodels.api as sm

```

## 4.2 Residual Time Plot


Residual plots help detect:
- remaining structure (runs, cycles)
- outliers (unexpected spikes)
- variance changes (heteroskedasticity)


In [ ]:
stl_male = STL(male, period=12).fit()
stl_female = STL(female, period=12).fit()

plt.figure(figsize=(10,3))
plt.plot(stl_male.resid)
plt.title("STL Residuals (Male)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,3))
plt.plot(stl_female.resid)
plt.title("STL Residuals (Female)")
plt.tight_layout()
plt.show()



## 5. Lag-1 Scatter After Seasonality Removal

### Why?
To test whether lag-1 dependence was mainly seasonal.


In [ ]:
lag_scatter(pd.Series(male_harm_resid), "Lag-1 Scatter (Male residuals after harmonic removal)")
lag_scatter(pd.Series(female_harm_resid), "Lag-1 Scatter (Female residuals after harmonic removal)")


### Interpretation
If the diagonal structure shrinks vs the original lag plot → much of the dependence was seasonal.
If a diagonal remains → stochastic persistence beyond seasonality remains.


## 6. STL Decomposition (Flexible Seasonality)


Harmonic regression assumes a fixed sinusoidal shape.
STL can capture changing seasonal shapes/amplitudes.


In [ ]:
stl_male = STL(male, period=12).fit()
stl_female = STL(female, period=12).fit()

plt.figure(figsize=(10,3))
plt.plot(stl_male.resid)
plt.title("STL Residuals (Male)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,3))
plt.plot(stl_female.resid)
plt.title("STL Residuals (Female)")
plt.tight_layout()
plt.show()


### Interpretation
If STL residuals look more random than harmonic residuals, STL removed seasonality more effectively.
But we still need formal diagnostics.


### Interpretation of output
Your STL residual plot highlights notable deviations (e.g., around 1976 and 1977), suggesting:
- occasional shocks/outliers beyond smooth seasonality
- potential remaining autocorrelation

We still need formal tests (ACF + Ljung–Box).


## Autocovariance

### Mathematical Definition

For a weakly stationary time series $ X_t $ with mean $ \mu = E[X_t] $,

$
\gamma(k) = \text{Cov}(X_t, X_{t-k})
$

Equivalently,

$
\gamma(k) = E[(X_t - \mu)(X_{t-k} - \mu)]
$

### What Does This Formula Say?

- If both $X_t$ and $X_{t-k}$ are above the mean → product positive  
- If both are below the mean → product positive  
- If one is above and one below → product negative  

So:

- $ \gamma(k) > 0 $ → persistence  
- $ \gamma(k) < 0 $ → oscillation  
- $ \gamma(k) = 0 $ → no linear dependence  

### Statistical Meaning

Autocovariance measures how much the present value moves together with its past.

It quantifies **memory in the time series**.

### Biological Interpretation

If $X_t$ represents monthly respiratory deaths:

- $ \gamma(1) > 0 $ means high mortality this month tends to be followed by high mortality next month → epidemic persistence.

- $ \gamma(12) > 0 $ means this January resembles last January → seasonal recurrence.

Autocovariance measures biological memory across time.

## Autocorrelation Function (ACF)

### Mathematical Definition

$
\rho(k) = \frac{\gamma(k)}{\gamma(0)}
$

where

$
\gamma(0) = \text{Var}(X_t)
$

### What Does This Mean?

Autocorrelation is just normalized autocovariance.

Properties:

- $ \rho(0) = 1 $
- $ -1 \le \rho(k) \le 1 $

It measures correlation between present and past values.

### Statistical Meaning

ACF tells us:

How strongly does the current value depend on values k steps back?

- Slow decay → long memory  
- Fast drop → short memory  
- Seasonal spikes → recurring yearly dependence  

### Biological Interpretation

If $ \rho(1) = 0.6 $:

Current month’s deaths are strongly linked to last month’s deaths.

In epidemics:

- Transmission chains create short-term dependence.
- Hospital load depends on recent hospital load.

ACF measures transmission persistence.

## 7. White Noise Diagnostics (ACF + Ljung–Box)

### Why?
A good model should leave residuals that are approximately white noise.

We use:
- ACF plot (visual)
- Ljung–Box test : a Ljung-Box test is run to determine whether the autocorrelations of the residuals at different lags are statistically significant.

Ljung–Box $H_0$: residuals have no autocorrelation up to the chosen lag.


In [ ]:
plot_acf(male_harm_resid, lags=24)
plt.title("ACF: Male harmonic residuals")
plt.show()

acorr_ljungbox(male_harm_resid, lags=[12], return_df=True)


### Interpretation
- ACF spikes outside band → autocorrelation remains.
- Ljung–Box p < 0.05 → reject white noise → dependence remains.
This motivates SARIMA modeling.


### Interpretation
After seasonal differencing, the mean is often more stable.
If stationarity improves, we can model remaining short-lag dependence with ARMA terms.


In [ ]:
plot_acf(male, lags=36)
plt.title("ACF: Male ")
plt.show()

## Partial Autocorrelation Function (PACF)

### Mathematical Definition

PACF at lag k is the last coefficient in the regression:

$
X_t = \phi_{k1}X_{t-1} + \dots + \phi_{kk}X_{t-k} + \varepsilon_t
$

The PACF value equals $ \phi_{kk} $.

### What Does This Mean?

PACF measures:

The direct effect of lag k after removing the effects of lags 1,...,k−1.

ACF includes both direct and indirect effects.
PACF isolates the direct influence.

### Statistical Meaning

If PACF cuts off after lag p:

### Biological Interpretation

If PACF cuts off after lag 1:

Only the previous month directly influences the current month.

Two months ago matters only indirectly through last month.

PACF detects the direct biological influence window.

## 9.1 PACF 



In [ ]:

plot_pacf(male_harm_resid, lags=12, method='ywadjusted')
plt.title("PACF: Male harmonic residuals")
plt.show()


# How to Read ACF and PACF Plots



---

## 1. What Is Plotted on the Axes?

### Horizontal axis (x-axis):
Lag k

- Lag 1 → relationship with previous time step
- Lag 2 → relationship with two time steps ago
- Lag 12 (monthly data) → relationship with same month last year

### Vertical axis (y-axis):
Autocorrelation value

$
\rho(k) = \text{Corr}(X_t, X_{t-k})
$

Each vertical bar represents the sample autocorrelation at lag k.

---

## 2. What Do the Blue Confidence Bands Mean?

The dashed horizontal lines represent approximately:

$
\pm \frac{2}{\sqrt{n}}
$

If a bar exceeds these bounds:

→ The correlation is statistically different from zero (approximately at 5% level).

Important:
A few random spikes are normal.
Consistent patterns are meaningful.

---

## 3. Reading the ACF Plot

The ACF shows total correlation between present and past.

### What to look for:

### (A) Slow geometric decay
- Bars gradually shrink toward zero
→ Suggests AR-type behavior

### (B) Sharp cutoff after lag q
- Significant at lags 1,...,q
- Then near zero afterwards
→ Suggests MA(q)

### (C) Seasonal spikes
- Large spikes at lag 12, 24, 36
→ Suggests seasonal structure


## 4. Biological Interpretation

ACF and PACF measure biological memory.

In respiratory mortality:

- Lag 1 correlation:
  Transmission persistence across months.

- Lag 2–3 correlation:
  Short-term epidemic dynamics.

- Lag 12 correlation:
  Winter-to-winter similarity.

If correlations decay slowly:
The epidemic shock fades gradually.

If correlations cut off sharply:
Shocks have limited duration.

---

## 5. Important Warning

ACF/PACF patterns are guides — not guarantees.

Always:
1. Propose a model.
2. Fit it.
3. Check residual ACF.
4. Use Ljung–Box test.

Diagnostics determine model adequacy.

# Detecting Dominant Periodicity from Raw Time Series

In many real-world problems, we do **not know the seasonal period in advance**.

Instead of assuming a period (e.g., 12 for monthly data), we can detect it directly from the data using frequency-domain methods.

We will use two  approaches:

1. Periodogram (scipy.signal)
2. Fast Fourier Transform (FFT)



## Step 0: Preprocessing

Before computing spectral quantities:

- Remove missing values
- Subtract the mean (to remove the zero-frequency component)
- Confirm sampling interval

For monthly data, sampling interval = 1 month.

In [ ]:

# Remove missing values
y = male.dropna().values

# Remove mean (important for spectral analysis)
y = y - np.mean(y)

n = len(y)
sampling_interval = 1  # 1 month

## Method 1: Periodogram (scipy.signal)

The periodogram estimates the spectral density:

$
P(f) = \left| \sum_{t=0}^{n-1} X_t e^{-2\pi i f t} \right|^2
$

The frequency with the highest power corresponds to the strongest repeating cycle.

The period is:

$
\text{Period} = \frac{1}{\text{Frequency}}
$

Since our sampling is monthly, the detected period will be measured in months.

In [ ]:
from scipy.signal import periodogram


# Compute periodogram
freqs, power = periodogram(y, fs=1)  # fs=1 sample per month

# Remove zero frequency (mean component)
freqs = freqs[1:]
power = power[1:]

# Identify dominant frequency
dominant_freq = freqs[np.argmax(power)]
dominant_period = 1 / dominant_freq

print("Dominant frequency:", dominant_freq)
print("Dominant period (months):", dominant_period)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(freqs, power)
plt.xlabel("Frequency (cycles per month)")
plt.ylabel("Power")
plt.title("Periodogram – Male Mortality")
plt.show()

## Method 2: Fast Fourier Transform (FFT)

The discrete Fourier transform decomposes the time series into sinusoidal components:

$
X_t = \sum_{k=0}^{n-1} c_k e^{2\pi i k t / n}
$

The magnitude of each coefficient indicates the strength of that frequency component.

We compute the squared magnitude:

$
\text{Power} = |FFT|^2
$

In [ ]:
from scipy.fft import fft, fftfreq

# Compute FFT
yf = fft(y)
xf = fftfreq(n, d=1)  # monthly spacing

# Keep only positive frequencies
positive = xf > 0
xf = xf[positive]
power_fft = np.abs(yf[positive])**2

dominant_freq_fft = xf[np.argmax(power_fft)]
dominant_period_fft = 1 / dominant_freq_fft

print("FFT dominant frequency:", dominant_freq_fft)
print("Dominant period (months):", dominant_period_fft)